# Monthly Card Data Processing

This notebook processes 12 months of scraped card transaction data from a website. It filters irrelevant columns, unmerges cells, aggregates volume and value per transaction mode, removes specific banks, tags remaining banks, and cleans empty rows/columns.

In [37]:
# importing required libraries
import os
import pandas as pd

In [38]:
# function to perform necessary cleaning
def clean_and_save_df(df, output_name, output_directory):
    # Rename columns to A to AC as in Excel
    df.columns = [chr(ord('A') + i // 26 - 1) + chr(ord('A') + i % 26) if i >= 26 else chr(ord('A') + i) for i in range(29)]

    # Drop top 9 rows
    df = df.drop(df.index[:9]).reset_index(drop=True)

    # Get the starting index for deleting bottom rows from payment banks
    del_from_payment_banks_index = df.index[df['B'].str.contains('Payment Banks', na=False)].tolist()

    # Find the maximum index in the list
    max_index = max(del_from_payment_banks_index, default=-1)

    # Replace df with the subset of rows excluding del_from_index and all rows below it
    df = df.loc[:max_index].reset_index(drop=True)

    # Remove rows that had only side heading for bank type
    df = df.dropna(thresh=df.shape[1] - 5).reset_index(drop=True)

    # To remove credit cards - Cash Withdrawal and other Infrastructure data
    columns_to_delete = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'R', 'S']
    df = df.drop(columns=columns_to_delete, axis=1)

    # To remove debit cards - Cash Withdrawal data
    df = df.iloc[:, :-4]

    # Rename columns to follow a standardized naming convention for better clarity and consistency
    df.columns = ['bank_name', 'credit_cards', 'debit_cards', 'cc_pos_vol(in actuals)',
                  'cc_pos_value(in 1000Rs)', 'cc_online_vol(in actuals)', 'cc_online_value(in 1000Rs)',
                  'cc_others_vol(in actuals)', 'cc_others_value(in 1000Rs)', 'dc_pos_vol(in actuals)',
                  'dc_pos_value(in 1000Rs)', 'dc_online_vol(in actuals)', 'dc_online_value(in 1000Rs)',
                  'dc_others_vol(in actuals)', 'dc_others_value(in 1000Rs)']

    # Lists of bank type and their corresponding banks
    # List of Public Sector Banks
    public_sector_banks = [
        'BANK OF BARODA', 'BANK OF INDIA', 'BANK OF MAHARASHTRA', 'CANARA BANK', 
        'CENTRAL BANK OF INDIA', 'INDIAN BANK', 'INDIAN OVERSEAS BANK', 'PUNJAB AND SIND BANK', 
        'PUNJAB NATIONAL BANK', 'STATE BANK OF INDIA', 'UCO BANK', 'UNION BANK OF INDIA'
    ]

    # List of Private Sector Banks
    private_sector_banks = [
        'AXIS BANK LTD', 'BANDHAN BANK LTD', 'CATHOLIC SYRIAN BANK LTD', 'CITY UNION BANK', 
        'CITY UNION BANK LTD.', 'CSB BANK LTD', 'CSB BANK LTD.', 'DCB BANK LTD',
        'DHANALAKSHMI BANK LTD', 'FEDERAL BANK LTD', 'HDFC BANK LTD', 'ICICI BANK LTD', 
        'IDBI BANK LTD', 'IDBI LTD', 'IDFC Bank Limited', 'IDFC FIRST BANK LTD', 'INDUSIND BANK LTD', 
        'JAMMU AND KASHMIR BANK', 'JAMMU AND KASHMIR BANK LTD', 'KARNATAKA BANK LTD', 'KARUR VYSYA BANK LTD', 
        'KOTAK MAHINDRA BANK LTD', 'NAINITAL BANK LTD', 'RATNAKAR BANK LIMITED', 'RBL BANK LTD', 'SOUTH INDIAN BANK', 
        'TAMILNAD MERCANTILE BANK LTD', 'YES BANK LTD'
    ]

    # List of Foreign Banks
    foreign_banks = [
        'AMERICAN EXPRESS','AMERICAN EXPRESS BANKING CORPORATION','BANK OF AMERICA','BARCLAYS BANK PLC',
        'CITI BANK','DBS BANK','DBS INDIA BANK LTD','DEUTSCHE BANK LTD','HONGKONG AND SHANGHAI BKG CORPN',
        'HSBC LTD','SBM Bank India','SBM BANK INDIA LTD','STANDARD CHARTERED BANK LTD','WOORI BANK'
    ]

    # Assign Bank Type based on bank names
    df.loc[df['bank_name'].isin(public_sector_banks), 'Bank Type'] = 'Public Sector Banks'
    df.loc[df['bank_name'].isin(private_sector_banks), 'Bank Type'] = 'Private Sector Banks'
    df.loc[df['bank_name'].isin(foreign_banks), 'Bank Type'] = 'Foreign Banks'

    # Reorder columns
    df = df[['bank_name', 'Bank Type', 'credit_cards', 'debit_cards', 'cc_pos_vol(in actuals)',
             'cc_pos_value(in 1000Rs)', 'cc_online_vol(in actuals)', 'cc_online_value(in 1000Rs)',
             'cc_others_vol(in actuals)', 'cc_others_value(in 1000Rs)', 'dc_pos_vol(in actuals)',
             'dc_pos_value(in 1000Rs)', 'dc_online_vol(in actuals)', 'dc_online_value(in 1000Rs)',
             'dc_others_vol(in actuals)', 'dc_others_value(in 1000Rs)']]

    # Save the DataFrame as a CSV file
    output_file = os.path.join(output_directory, f"{output_name}.csv")

    while os.path.exists(output_file):
        output_file = os.path.join(output_directory, f"{output_name}.csv")

    df.to_csv(output_file, index=False)
    print(f"DataFrame saved as: {output_file}")

In [40]:
# source and destination directories
source_dir = "cleaning_data/"
destination_dir = "cleaned_data/"
# List all Excel files in the source directory
excel_files = [file for file in os.listdir(source_dir) if file.endswith('.XLSX')]
# Loop through each Excel file
for excel_file in excel_files:
    # Construct the full file path
    excel_path = os.path.join(source_dir, excel_file)

    # Read Excel file into a DataFrame
    df = pd.read_excel(excel_path)

    # Extract the file name without extension for output
    output_name = os.path.splitext(excel_file)[0]

    # Call the processing function
    clean_and_save_df(df, output_name, destination_dir)

DataFrame saved as: cleaned_data/2022APRIL.csv
DataFrame saved as: cleaned_data/2022AUGUST.csv
DataFrame saved as: cleaned_data/2022DECEMBER.csv
DataFrame saved as: cleaned_data/2022JULY.csv
DataFrame saved as: cleaned_data/2022JUNE.csv
DataFrame saved as: cleaned_data/2022MAY.csv
DataFrame saved as: cleaned_data/2022NOVEMBER.csv
DataFrame saved as: cleaned_data/2022OCTOBER.csv
DataFrame saved as: cleaned_data/2022SEPT.csv
DataFrame saved as: cleaned_data/2023FEBRUARY.csv
DataFrame saved as: cleaned_data/2023JAN.csv
DataFrame saved as: cleaned_data/2023MARCH.csv


___